# Homework 03. Jeun Hyun (Student ID -20191328)


In [1]:
# The function to read sequences from a FASTA file
def read_fasta(tmp_filename):
    rv = dict()
    f = open(tmp_filename, 'r')
    for line in f:
        if line.startswith('>'):
            tmp_h = line.strip().lstrip('>')
            rv[tmp_h] = ''
        else:
            rv[tmp_h] += line.strip().replace(' ', '')
    f.close()
    return rv

In [2]:
# Read GCF_009858895.2 genomic info.
filename_genomes = '../data/GCF_009858895.2_ASM985889v3_genomic.fna'
genome_list = read_fasta(filename_genomes)
genome_seq = list(genome_list.values())[0]

print('GCF_009858895.2 genomic sequence is', genome_seq[:25], '...', genome_seq[-25:])

GCF_009858895.2 genomic sequence is ATTAAAGGTTTATACCTTCCCAGGT ... AAAAAAAAAAAAAAAAAAAAAAAAA


In [3]:
# Make N-protein cDNA sequence.
f = open('../data/GCF_009858895.2_ASM985889v3_genomic.gff', 'r')

for line in f:
    try:
        if 'Name=N' in line.split()[8]:
            N_genomic = genome_seq[int(line.split()[3])-1 : int(line.split()[4])]
            print('N-protein cDNA sequence is', N_genomic[:25], '...', N_genomic[-25:])
    except:
        continue
        
f.close()

N-protein cDNA sequence is ATGTCTGATAATGGACCCCAAAATC ... CAGTGCTGACTCAACTCAGGCCTAA


In [4]:
# Translate it to protein sequence. 
f = open("N_genomic.fa", 'w')
data = "> N_genomic\n%s\n" % N_genomic
f.write(data)
f.close()

# (terminal) ~/git/BME203-2020/03_Annotation$ transeq --sequence N_genomic.fa --outseq N_trans.fa

N_protein = list(read_fasta('N_trans.fa').values())[0][:-1] # exclude not-designated codon - stop codon
print('N-protein sequence is', N_protein[:25], '...', N_protein[-25:]) 

N-protein sequence is MSDNGPQNQRNAPRITFGGPSDSTG ... LPAADLDDFSKQLQQSMSSADSTQA


In [5]:
# Q1. Compare it to GCF_009858895.2_ASM985889v3_protein.faa
f = open('../data/GCF_009858895.2_ASM985889v3_protein.faa', 'r')
protein_list = read_fasta('../data/GCF_009858895.2_ASM985889v3_protein.faa')
protein_h = list(protein_list.keys())
protein_seq = list(protein_list.values())

detect = 0
for i in range(len(protein_h)):
    if 'nucleocapsid phosphoprotein' in protein_h[i]:
        detect = 1
        if N_protein[:-1] == protein_seq[i]:
            print('The translated sequence and the sequence in data of N_protein(nucleocapsid phosphoprotein) are identical')
        else:
            print('The translated sequence and the sequence in data of N_protein(nucleocapsid phosphoprotein) are not identical')
if detect == 0:
    print('Cannot find N_protein(nucleocapsid phosphoprotein) sequence in data')
        
f.close()

The translated sequence and the sequence in data of N_protein(nucleocapsid phosphoprotein) are not identical


In [6]:
# Based on this info, extract N proteins cDNA/proteins from other SARS-CoV2 genomes (03_27).

# in cDNA
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype nucl -in ../data/2019nCoV_genomes.2020_03_27.fa -out 2019nCoV_genomes.2020_03_27
# (terminal) ~/git/BME203-2020/03_Annotation$ blastn -db 2019nCoV_genomes.2020_03_27 -query N_genomic.fa -outfmt 6 -out 2019nCoV_genomes.2020_03_27_blastn.out

f = open('2019nCoV_genomes.2020_03_27_blastn.out', 'r')

genome_list = read_fasta('../data/2019nCoV_genomes.2020_03_27.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()


# in protein
# (terminal) ~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/2019nCoV_genomes.2020_03_27.fa --outseq ../data/2019nCoV_genomes.2020_03_27_trans6.fa -frame 6
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in ../data/2019nCoV_genomes.2020_03_27_trans6.fa -out ../data/2019nCoV_genomes.2020_03_27_trans6
# (terminal) ~/git/BME203-2020/03_Annotation$ blastp -db ../data/2019nCoV_genomes.2020_03_27_trans6 -query N_trans.fa -outfmt 6 -out 2019nCoV_genomes.2020_03_27_trans6_blastp.out

f = open('2019nCoV_genomes.2020_03_27_trans6_blastp.out', 'r')

genome_list = read_fasta('../data/2019nCoV_genomes.2020_03_27.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()

In [7]:
# Q2. Which pairs are most similar ones? Which pairs are most different ones?

# in cDNA
print('In terms of cDNA')
try:
    f = open('2019nCoV_genomes.2020_03_27_blastn.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))

    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))

    f.close()
except:
    print('blast fail')

print('\n')

# in protein
print('In terms of protein')
try:
    f = open('2019nCoV_genomes.2020_03_27_trans6_blastp.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))

    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))


    f.close()
except:
    print('blast fail')


In terms of cDNA
MT049951 MT066176 MT066175 MT072688 MT093571 MT093631 MT123292 LC528232 LC528233 MT126808 MT135044 MT135042 MT135041 MT152824 MT050493 MT012098 MT066156 MT121215 MT163718 MT163717 MT163716 MT163719 MT192773 MT192759 MT192765 MT192772 MT240479 MT246490 MT246489 MT246488 MT246487 MT246486 MT246484 MT246482 MT246481 MT246480 MT246479 MT246478 MT246477 MT246476 MT246475 MT246474 MT246473 MT246472 MT246471 MT246469 MT246468 MT246467 MT246466 MT246464 MT246462 MT246461 MT246460 MT246459 MT246458 MT246457 MT246455 MT246453 MT246452 MT246450 MT246449 are the most similar ones by 100.000000 percent identity
MT233522 MT246470 MT246451 are the most different ones by 99.762000 percent identity


In terms of protein
MT049951_2 MT066176_2 MT066175_2 MT072688_2 MT093571_2 MT093631_1 MT123293_2 MT123291_2 MT123292_2 LC528232_2 LC528233_1 MT126808_2 MT135044_2 MT135042_2 MT135041_2 MT152824_3 MT050493_3 MT012098_1 MT066156_2 MT121215_2 MT163718_2 MT163717_2 MT163716_2 MT163719_2 MT1927

In [8]:
# Based on this info, extract N proteins cDNA/proteins from other SARS & MERS genomes.

# SARS
# in cDNA
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype nucl -in ../data/SARS_genomes.2020_02_03.fa -out SARS_genomes.2020_02_03
# (terminal) ~/git/BME203-2020/03_Annotation$ blastn -db SARS_genomes.2020_02_03 -query N_genomic.fa -outfmt 6 -out SARS_genomes.2020_02_03_blastn.out

f = open('SARS_genomes.2020_02_03_blastn.out', 'r')

genome_list = read_fasta('../data/SARS_genomes.2020_02_03.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()


# in protein
# (terminal) ~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/SARS_genomes.2020_02_03.fa --outseq ../data/SARS_genomes.2020_02_03_trans6.fa -frame 6
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in ../data/SARS_genomes.2020_02_03_trans6.fa -out ../data/SARS_genomes.2020_02_03_trans6
# (terminal) ~/git/BME203-2020/03_Annotation$ blastp -db ../data/SARS_genomes.2020_02_03_trans6 -query N_trans.fa -outfmt 6 -out SARS_genomes.2020_02_03_trans6_blastp.out

f = open('SARS_genomes.2020_02_03_trans6_blastp.out', 'r')

genome_list = read_fasta('../data/SARS_genomes.2020_02_03.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()


# MERS
# in cDNA
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype nucl -in ../data/MERS_genomes.2020_02_03.fa -out MERS_genomes.2020_02_03
# (terminal) ~/git/BME203-2020/03_Annotation$ blastn -db MERS_genomes.2020_02_03 -query N_genomic.fa -outfmt 6 -out MERS_genomes.2020_02_03_blastn.out

f = open('MERS_genomes.2020_02_03_blastn.out', 'r')

genome_list = read_fasta('../data/MERS_genomes.2020_02_03.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()


# in protein
# (terminal) ~/git/BME203-2020/03_Annotation$ transeq --sequence ../data/MERS_genomes.2020_02_03.fa --outseq ../data/MERS_genomes.2020_02_03_trans6.fa -frame 6
# (terminal) ~/git/BME203-2020/03_Annotation$ makeblastdb -dbtype prot -in ../data/MERS_genomes.2020_02_03_trans6.fa -out ../data/MERS_genomes.2020_02_03_trans6
# (terminal) ~/git/BME203-2020/03_Annotation$ blastp -db ../data/MERS_genomes.2020_02_03_trans6 -query N_trans.fa -outfmt 6 -out MERS_genomes.2020_02_03_trans6_blastp.out

f = open('MERS_genomes.2020_02_03_trans6_blastp.out', 'r')

genome_list = read_fasta('../data/MERS_genomes.2020_02_03.fa')
for line in f:
    for genome in list(genome_list.keys()):
        if line.split()[1] in genome:
            print(end='')
#            print(genome_list[genome][int(line.split()[8])-1:int(line.split()[9])], 'from', genome.split()[0])
    
f.close()

In [9]:
# Q3. Which protein is most similar to SARS-CoV N protein? Which protein is most different to MERS N protein?
# SARS
print('For SARS')
# in cDNA
print('in terms of cDNA,')
try:
    f = open('SARS_genomes.2020_02_03_blastn.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))

    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))

    f.close()
except:
    print('blast fail')

print('\n')

# in protein
print('in terms of protein,')
try:
    f = open('SARS_genomes.2020_02_03_trans6_blastp.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))

    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))

    f.close()
except:
    print('blast fail')

print('\n\n')


# MERS
print('For MERS')
# in cDNA
print('in terms of cDNA,')
try:
    f = open('MERS_genomes.2020_02_03_blastn.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))
        
    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))
    
    f.close()
except:
    print('blast fail')

print('\n')

# in protein
print('in terms of protein,')
try:
    f = open('MERS_genomes.2020_02_03_trans6_blastp.out', 'r')

    subject_list = []
    identity_list = []
    for line in f:
        subject_list.append(line.split()[1])
        identity_list.append(float(line.split()[2]))

    for i in range(len(identity_list)):
        if identity_list[i] == max(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most similar ones by %f percent identity' % max(identity_list))

    for i in range(len(identity_list)):
        if identity_list[i] == min(identity_list):
            print(subject_list[i], end = ' ')
    print('are the most different ones by %f percent identity' % min(identity_list))

    f.close()
except:
    print('blast fail')

For SARS
in terms of cDNA,
ENA|DQ084200|DQ084200.1 ENA|DQ084199|DQ084199.1 ENA|DQ022305|DQ022305.2 are the most similar ones by 88.862000 percent identity
ENA|AY686863|AY686863.1 are the most different ones by 88.217000 percent identity


in terms of protein,
KU973692.1_3 DQ898174.1_1 FJ882949.1_1 FJ882934.1_3 FJ882937.1_2 FJ882935.1_1 FJ882933.1_2 FJ882947.1_3 FJ882946.1_1 FJ882927.1_2 FJ882932.1_3 FJ882939.1_3 FJ882936.1_3 FJ882938.1_2 FJ882941.1_1 FJ882929.1_1 FJ882944.1_3 FJ882940.1_3 FJ882954.1_2 FJ882960.1_3 FJ882955.1_3 FJ882931.1_3 FJ882928.1_2 FJ882926.1_2 FJ882930.1_2 AY362698.1_1 AY394850.2_2 are the most similar ones by 90.544000 percent identity
DQ084200.1_3 DQ022305.2_2 are the most different ones by 89.573000 percent identity



For MERS
in terms of cDNA,
blast fail


in terms of protein,
KX574227.1_3 are the most similar ones by 50.142000 percent identity
KX108941.1_2 are the most different ones by 43.333000 percent identity
